In [1]:
# !pip install pymed
# ! pip install nltk
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# !pip install biopython
# !pip install ahocorapy
# !pip install pandas
# !pip install wordcloud
# !pip install matplotlib

In [10]:
import json
from Bio import Entrez

def related_keywords_mesh(papers):
    mesh_kewords_related = {}
    for paper in papers:
        mesh_kewords_related[paper['doi']] = paper['mesh_terms']
    return mesh_kewords_related

In [11]:
import json
import dateutil.parser

# custom Decoder
def DecodeDateTime(Dict):
    if 'publication_date' in Dict and Dict["publication_date"]!=None:
        Dict["publication_date"] = dateutil.parser.parse(Dict["publication_date"])
        return Dict
    else:
        return Dict

# use of object_hook
with open('cache_articles.json', 'r') as f:
    decoded_cache_articles = json.load(f,object_hook=DecodeDateTime)

In [348]:
domain_mesh.keys()

dict_keys(['affinity', 'membrane', 'TFF', 'continuous processing', 'virus capsid', 'virus-like particles', 'bioconjugation', '5MP', '5MPs', 'crystalization', 'aptamer', 'photoresponsive', 'HCIC', 'thermoresponsive polymer', 'covalent capture', 'ionic liquids', 'liquid-liquid separation', 'Astrea', 'Prometic', 'ELP', 'affinity precipitation', 'Protein A', 'PEG', 'polymer precipitation', 'ATPS', 'magnetic particles', 'droplet forming protein', 'intein', 'IgG structure', 'Protein L', 'charged membrane', 'peptoid', 'PVA', 'PVP', 'monolith', 'novel support', 'VHH', 'precipitation', 'Protein G', 'PAA', 'expanded bed', 'ion exchange', 'membrane chromatography', 'supercritical fluid', 'electrophoresis', 'Gradipore', 'recombinant IgG', 'cyclic peptides', 'Chreto'])

In [20]:
import json
with open('train_dataset_mesh.json','r') as f:
    train_dataset_mesh = json.load(f)

In [21]:
import json
# save domain mesh
with open('domain_mesh.json','r') as f:
    domain_mesh = json.load(f)

In [351]:
train_keywords = list(train_dataset_mesh.keys())

In [352]:
global_domain_mesh = []
for keyword in train_keywords:
    for key in train_dataset_mesh[keyword]:
        if key not in global_domain_mesh:
            global_domain_mesh.append(key)

len(global_domain_mesh)

115

# ranking

In [353]:
## TEST

# virus capsid;virus-like particles;Protein A
# PEG;precipitation
# IgG structure
# affinity;monolith
# affinity;precipitation
# ionic liquids

In [354]:
## TRAIN

# affinity
# affinity;intein
# affinity;Protein A
# affinity;Protein A;electrophoresis
# affinity;Protein A;Protein L
# affinity;VHH
# bioconjugation;5MPs
# ELP
# membrane
# membrane;Protein L
# Protein A

In [13]:
search_term = "Protein A"
mode = "train"

In [14]:
searchterms = search_term.split(";")
print(searchterms)
search_database = []

for st in searchterms:
    if st in decoded_cache_articles:
        search_database.extend(decoded_cache_articles[st])

['Protein A']


In [15]:
len(search_database)

5417

In [16]:
mesh_keywrds_extracted = related_keywords_mesh(search_database)

In [17]:
len(mesh_keywrds_extracted)

1772

In [22]:
if mode == "train":
    new_search_vector = []
    for st in searchterms:
        for wrd in train_dataset_mesh[st]:
            if wrd not in new_search_vector:
                new_search_vector.append(wrd)
else:
    new_search_vector = global_domain_mesh.copy()
    for st in searchterms:
        for wrd in domain_mesh[st]:
            if wrd not in new_search_vector:
                new_search_vector.append(wrd)

In [23]:
len(new_search_vector)

38

In [24]:
from collections import Counter
rank = {}
# target_mesh = [val[0] for val in train_dataset_mesh[term_search]]
# target_mesh = [val for val in domain_mesh[term_search]]
target_mesh = new_search_vector
for key,value in mesh_keywrds_extracted.items():
    count = 0
    for val in value:
        if val[0] in target_mesh:
            count+=1
    rank[key] = count/len(target_mesh)
    
dict(sorted(Counter(rank).items(), key=lambda item: item[1],reverse=True))

{'S0021-9673(20)30515-X': 0.2894736842105263,
 'S0021-9673(19)30571-0': 0.2631578947368421,
 'E161': 0.2631578947368421,
 '10.1371/journal.pone.0025282': 0.2631578947368421,
 '10.1371/journal.pone.0056597': 0.2631578947368421,
 '10.1016/j.chroma.2010.06.029': 0.2631578947368421,
 'S0021-9673(18)30188-2': 0.23684210526315788,
 '10.1002/jssc.201500178': 0.23684210526315788,
 'S0021-9673(20)31117-1': 0.21052631578947367,
 '10.1101/pdb.top098509': 0.21052631578947367,
 'S1570-0232(19)31081-5': 0.21052631578947367,
 '10.1002/bit.27213': 0.21052631578947367,
 '10.1016/j.pep.2014.09.011': 0.21052631578947367,
 'S1046-5928(20)30353-3': 0.18421052631578946,
 'S0022-1759(20)30091-0': 0.18421052631578946,
 'S0731-7085(19)30463-7': 0.18421052631578946,
 'S1570-0232(18)31738-0': 0.18421052631578946,
 '10.1007/978-1-4939-9145-7_8': 0.18421052631578946,
 '10.1039/c8bm01220b': 0.18421052631578946,
 'S1046-5928(18)30561-8': 0.18421052631578946,
 '10.1002/jmr.2223': 0.18421052631578946,
 '10.1016/j.tala

In [25]:
# affinity  - '10.1002/biot.201600357' 71
# affinity;intein - '10.2174/1389203720666190208110416' 0
# affinity;Protein A - '10.3390/ma9120994' 3724
# affinity;Protein A;electrophoresis - '10.1371/journal.pone.0139137' 0
# affinity;Protein A;Protein L - '10.1080/19420862.2019.1662690' 305
# affinity;VHH - '10.1128/aem.02595-14' 14

# bioconjugation;5MPs - '10.1021/jacs.7b00670' 0

# ELP - '10.3389/fbioe.2019.00233' 685
# ELP - '10.1016/j.jconrel.2014.06.028' 9
# ELP - '10.1016/j.ab.2011.04.034' 51
# ELP - '10.1002/0471140864.ps0611s61' 33
# ELP - '10.1016/j.jconrel.2015.11.010' 4
# ELP - '10.1021/bm400167h' 0

# membrane - '10.1073/pnas.2000223117' 1
# membrane - '10.1016/j.jmb.2018.06.038' 3
# membrane;Protein L - '10.1038/s41467-018-05403-1' 0

# Protein A - '10.1371/journal.pone.0025282' 3
# Protein A - '10.1080/19420862.2019.1565749' 40

In [29]:
from collections import Counter
count=0
for key,value in dict(sorted(Counter(rank).items(), key=lambda item: item[1],reverse=True)).items():
#     print(key,value)
    if key =='10.1080/19420862.2019.1565749':
        print(count)
#     if count == 50:
#         break
    count+=1

40
